In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import tqdm

import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import copy
from torch.multiprocessing import Pipe

import gym
import caviar_tools
from beamselect_env import BeamSelectionEnv

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()
device = torch.device("cpu")

In [3]:
epi_val = [500,500]
gym_env_ind_val = list()
for i in range(epi_val[0],epi_val[1]+1):
    gym_env_ind_val.append(BeamSelectionEnv([i,i]))

n_steps_epi_val = list()
for i in range(epi_val[0],epi_val[1]+1):
    n_steps_epi_val.append(caviar_tools.linecount([i,i]))

# n_steps = sum(n_steps_epi)
n_steps_val = sum(n_steps_epi_val)

/home/mohit/anaconda3/envs/ITU/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
from agents_cur import *
from utils_cur import *

import numpy as np
import pickle

In [5]:
train_method = 'ICM'
env_id = None #BreakoutNoFrameskip-v4
env_type = 'beamselect'

In [6]:
model_path = './model_state_dict/model_curiosity'
icm_path = './model_state_dict/icm_curiosity'

In [7]:
lam = 0.95
num_worker = 1

num_step = int(512)

ppo_eps = float(0.1)
epoch = int(3)
mini_batch = int(8)
BATCH_SIZE = int(num_step * num_worker / mini_batch) #16
learning_rate = float(1e-4)
entropy_coef = float(0.001)
gamma = float(0.8)
eta = float(1)

clip_grad_norm = float(0.5)

pre_obs_norm_step = int(3000)#int(10000) Not much effect

HISTORY_SIZE = BATCH_SIZE*2
STATES_USED = 13

In [8]:
input_size = [BATCH_SIZE,STATES_USED]  
output_size = 192 #64*3

In [9]:
f = open('obs_rms.pkl', 'rb')
obs_rms = pickle.load(f)

In [10]:
agent = ICMAgent

agent = agent(
        input_size,
        output_size,
        num_worker,
        num_step,
        gamma,
        lam=lam,
        learning_rate=learning_rate,
        ent_coef=entropy_coef,
        clip_grad_norm=clip_grad_norm,
        epoch=epoch,
        batch_size=BATCH_SIZE,
        ppo_eps=ppo_eps,
        eta=eta,
        use_cuda=False,
        use_gae=False,
        use_noisy_net=False
    )

In [11]:
states = np.zeros([1, BATCH_SIZE, 1,STATES_USED])

In [12]:
agent.model.load_state_dict(torch.load(model_path))
agent.icm.load_state_dict(torch.load(icm_path))

<All keys matched successfully>

In [13]:
Transition = namedtuple('Transition',
                        ('state'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size:int=BATCH_SIZE):
        return random.sample(self.memory, BATCH_SIZE)

    def __len__(self):
        return len(self.memory)

In [14]:
history = ReplayMemory(HISTORY_SIZE)

for i in range(HISTORY_SIZE):
    history.push(np.zeros((STATES_USED, )))

In [15]:
def run(action,env:gym.envs,history:ReplayMemory):
    s, reward, done, info = env.step([action//64, action%64])
    # print(type(s), s.shape)
    history.push(s.astype(np.float))
    
    return [np.array(history.sample(BATCH_SIZE)), reward, done, done, reward]

In [18]:
def val(epi_val:list,gym_env_ind:list=None):
    
    with torch.no_grad():
        
        history = ReplayMemory(HISTORY_SIZE)
        for i in range(HISTORY_SIZE):
            history.push(np.zeros((STATES_USED, )))
            
        f = open('obs_rms.pkl', 'rb')
        obs_rms = pickle.load(f)
        
        rall = 0
        rd = False
        intrinsic_reward_list = []
        states = np.zeros([1, BATCH_SIZE, 1,STATES_USED])
        
        ovr_val = 0
        for episode in tqdm.notebook.tqdm(range(epi_val[1]-epi_val[0]+1),desc='Val'):
            if len(gym_env_ind) == 0:
                env =  BeamSelectionEnv([episode,episode])
            else:
                env = copy.deepcopy(gym_env_ind[episode])
                
            rall=0
            for steps in range(n_steps_epi_val[episode]):
                
                actions, value, policy = agent.get_action(
                    (states - obs_rms.mean) / np.sqrt(obs_rms.var)
                )
                
                next_states, rewards, dones, real_dones, log_rewards, next_obs = [], [], [],\
                    [], [], []
                
                for action in actions:
                    s, r, d, rd, lr = run(action,env,history)
                    rall += r
                    next_states.append(s)
                
                next_states = np.stack(next_states)
                states = next_states[:, :, :, :]
                
            print(f'Running avg for episode {epi_val[0]+episode} is {rall/steps}')
            ovr_val +=rall
        
        print(f'Overall mean reward is {ovr_val/n_steps_val} ')

In [ ]:
val(epi_val,gym_env_ind_val)

Val:   0%|          | 0/1 [00:00<?, ?it/s]